In [3]:
import pandas as pd
wd = "/Users/timvigers/Documents/Work/Viral Shah/JDRF TIR/"

In [4]:
# See data_clean.py for cleaning procedure
df = pd.read_csv(wd + "Data_Clean/analysis_data.csv")

In [55]:
fol = 'Case 034'
subject_id = [int(i) for i in fol.split() if i.isdigit()][0]
# Find summary and CSV files
files = os.listdir(wd + "Data_Raw/3. Data Collection/Cleaned Final Data/Cases_T1D+DR/" + fol)
csvs = [f for f in files if ".csv" in f]
summary = [f for f in files if "summary" in f.lower()][0]
summary = pd.read_excel(wd + "Data_Raw/3. Data Collection/Cleaned Final Data/Cases_T1D+DR/" + fol + "/" + summary,engine = 'openpyxl')
c = csvs[5]

In [57]:
# Get visit number
vis = c.split("_")[0]
vis = [int(i) for i in vis.split() if i.isdigit()][0]
# Two weeks of data leading up to visit
end_date = summary.loc[summary.iloc[:,1] == vis]["Office Visit Date"]
start_date = end_date - pd.to_timedelta(14, unit='d')
end_date = end_date.dt.strftime('%Y-%m-%d').values[0]
start_date = start_date.dt.strftime('%Y-%m-%d').values[0]
# Import CGM file
cgm = pd.read_csv(wd + "Data_Raw/3. Data Collection/Cleaned Final Data/Cases_T1D+DR/" + fol + "/" + c,low_memory = False)
# Get timestamp and glucose columns, format
if "Timestamp (YYYY-MM-DDThh:mm:ss)" in cgm.columns:
    cgm = cgm[["Timestamp (YYYY-MM-DDThh:mm:ss)","Glucose Value (mg/dL)"]]
elif cgm.shape[1] == 19:
    start = cgm.loc[cgm.iloc[:,2] == "Device Timestamp"].index[0]
    cgm.columns = cgm.iloc[start,:]
    cgm = cgm.iloc[start+1:,:]
    cgm = cgm[["Device Timestamp","Historic Glucose mg/dL"]]
elif cgm.shape[1] > 40:
    start = cgm.loc[cgm['Unnamed: 2'] == "Sensor"].index[0]
    cgm.columns = cgm.iloc[start+1,:]
    cgm = cgm.iloc[start+2:,:]
    cgm.reset_index(inplace = True)
    cgm["timestamp"] = cgm["Date"] + " " + cgm["Time"]
    cgm = cgm[["timestamp","Sensor Glucose (mg/dL)"]]
cgm.columns = ["timestamp","glucose"]
cgm.dropna(inplace = True)
# Glucose to numeric replace "high" and "low" with limits of detection
cgm["glucose"].replace("High",400,inplace = True)
cgm["glucose"].replace("Low",40,inplace = True)
cgm["glucose"] = pd.to_numeric(cgm["glucose"],errors='coerce')
# Format timestamp if necessary
cgm["timestamp"] = [t.replace("T", " ") for t in cgm["timestamp"]]
cgm["timestamp"] = [cal.parse(t)[0] for t in cgm["timestamp"]]
cgm["timestamp"] = [datetime(*t[:6]) for t in cgm["timestamp"]]
# Complete cases
cgm.dropna(inplace = True)
# Re-index
cgm.set_index("timestamp",inplace = True)
# Remove all but two weeks prior
cgm = cgm.loc[start_date:end_date]

In [61]:
cgm.to_csv("test.csv")

In [59]:
end_date

'2016-10-13'

In [60]:
start_date

'2016-09-29'

In [62]:
summary

,DOB,Visit No.,Office Visit Date,A1c,CGM type at visit,Unnamed: 5
0,2004-05-12,1.0,2019-03-25,10.2,Dexcom,NaN
1,NaT,2.0,2019-01-03,9.6,Dexcom,NaN
2,NaT,3.0,2018-10-08,8.0,Dexcom,NaN
3,NaT,4.0,2018-07-03,8.2,Dexcom,NaN
4,NaT,5.0,2018-04-16,8.2,Dexcom,NaN
5,NaT,6.0,2018-01-08,8.6,Dexcom,NaN
6,NaT,7.0,2017-10-09,8.1,Dexcom,NaN
7,NaT,8.0,2017-07-06,8.4,Dexcom,NaN
8,NaT,9.0,2017-04-13,8.2,Dexcom,NaN
9,NaT,10.0,2017-01-17,7.9,Dexcom,NaN


In [63]:
14*24*60

20160